In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

import sympy as sp
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal
from sympy.matrices import *

Declaring indexed variables.

In [2]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

Declaring variables and types.  Will need to find a more efficient way to do this.  It works for now.|

In [3]:
var_beta = ['betax','betay','betaz']

betaL = ixp.zerorank1(DIM=3)

for itr in range(len(directions)):
    betaL[itr] = IndexedBase('state_fab'+var_beta[itr])[i,j,k,var_beta[itr]]
    
betaL = np.array(betaL)
    
var_gam = [['gxx','gxy','gxz'],['gyx','gyy','gyz'],['gzx','gzy','gzz']]

gamLL = ixp.declarerank2('g','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gamLL[itr1][itr2] = IndexedBase('state_fab'+var_gam[itr1][itr2])[i,j,k,var_gam[itr1][itr2]]

gamLL = np.array(gamLL)

var_gaminv = [['ginvxx','ginvxy','ginvxz'],['ginvyx','ginvyy','ginvyz'],['ginvzx','ginvzy','ginvzz']]
        
gamUU = ixp.declarerank2('ginv','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        gamUU[itr1][itr2] = IndexedBase('state_fab'+var_gaminv[itr1][itr2])[i,j,k,var_gaminv[itr1][itr2]]
        
gamUU = np.array(gamUU)
        
var_K = [['Kxx','Kxy','Kxz'],['Kyx','Kyy','Kyz'],['Kzx','Kzy','Kzz'],]
        
KLL = ixp.declarerank2('K','sym01')
for itr1 in range(len(directions)):
    for itr2 in range(len(directions)):
        KLL[itr1][itr2] = IndexedBase('state_fab'+var_K[itr1][itr2])[i,j,k,var_K[itr1][itr2]]

KLL = np.array(KLL)

Making a list of all variable names to assign string replacements in AMReXcode output later.

In [4]:
var_names = var_beta
for iter1 in var_gam:
    var_names.extend(iter1)
for iter2 in var_gaminv:
    var_names.extend(iter2)
for iter3 in var_K:
    var_names.extend(iter3)
print(var_names)

['betax', 'betay', 'betaz', 'gxx', 'gxy', 'gxz', 'gyx', 'gyy', 'gyz', 'gzx', 'gzy', 'gzz', 'ginvxx', 'ginvxy', 'ginvxz', 'ginvyx', 'ginvyy', 'ginvyz', 'ginvzx', 'ginvzy', 'ginvzz', 'Kxx', 'Kxy', 'Kxz', 'Kyx', 'Kyy', 'Kyz', 'Kzx', 'Kzy', 'Kzz']


Constructing the Gamma{i,j,k} connection coefficients from the metric.

In [5]:
DcgLLL = ixp.zerorank3(3)
GammaULL = ixp.zerorank3(3)

for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            DcgLLL[itri][itrj][itrk] = Dc(gamLL[itri][itrj],directions[itrk])
            
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        for itrk in range(len(directions)):
            for itrl in range(len(directions)):
                GammaULL[itri][itrj][itrk] += 1/2*gamUU[itri][itrl]*(DcgLLL[itrj][itrl][itrk]+DcgLLL[itrk][itrl][itrj]-DcgLLL[itrj][itrk][itrl])

Constructing covariant derivatives of beta.

In [7]:
covbetaLL = ixp.zerorank2(3)
covbetaTransLL = ixp.zerorank2(3)
for itri in range(len(directions)):
    for itrj in range(len(directions)):
        covbetaLL[itri][itrj] = Dc(betaL[itrj],directions[itri])
        covbetaTansLL = Dc(betaL[itri],directions[itrj])
        for itrk in range(len(directions)):
            covbetaLL[itri][itrj] += GammaULL[itrk][itri][itrj]*betaL[itrk]
            covbetaTransLL[itri][itrj] += GammaULL[itrk][itrj][itri]*betaL[itrk]

Constructing the RHS of metric in ADM evolution equations.  Evolution eq for K to come later.

In [9]:
gam_RHS = KLL+covbetaLL+covbetaTransLL

Example AMReX code for advancing gam_xy component.  Easy to output other components.

In [14]:
AMReXcode(gam_RHS[0][1],var_names,True, var_gam[0][1])

'rhs_fab(i, j, k, Idx::gxy) = (0.5*((-state_fab(i, j - 1, k, Idx::gxx) + state_fab(i, j + 1, k, Idx::gxx))/(2*dx[1]) - (-state_fab(i - 1, j, k, Idx::gxy) + state_fab(i + 1, j, k, Idx::gxy))/(2*dx[0]) + (-state_fab(i - 1, j, k, Idx::gyx) + state_fab(i + 1, j, k, Idx::gyx))/(2*dx[0]))*state_fab(i, j, k, Idx::ginvxx) + 0.5*(-(-state_fab(i, j, k - 1, Idx::gxy) + state_fab(i, j, k + 1, Idx::gxy))/(2*dx[2]) + (-state_fab(i, j - 1, k, Idx::gxz) + state_fab(i, j + 1, k, Idx::gxz))/(2*dx[1]) + (-state_fab(i - 1, j, k, Idx::gyz) + state_fab(i + 1, j, k, Idx::gyz))/(2*dx[0]))*state_fab(i, j, k, Idx::ginvxz) + 0.25*(-state_fab(i - 1, j, k, Idx::gyy) + state_fab(i + 1, j, k, Idx::gyy))*state_fab(i, j, k, Idx::ginvxy)/dx[0])*state_fab(i, j, k, Idx::betax) + (0.5*((-state_fab(i, j - 1, k, Idx::gxx) + state_fab(i, j + 1, k, Idx::gxx))/(2*dx[1]) - (-state_fab(i - 1, j, k, Idx::gxy) + state_fab(i + 1, j, k, Idx::gxy))/(2*dx[0]) + (-state_fab(i - 1, j, k, Idx::gyx) + state_fab(i + 1, j, k, Idx::gyx))/(2*